# LLM Endpoint Pentesting APIs

- Extract Project ID
- Create LLM Endpoint
- Retrieve Pentest Template ID
- Run Pentest
- Wait for Pentest to Complete
- Download CSV report

### Extract Project ID

#### Setting up API Access
Retrieve from Admin Console
- API key
- customer ID

![](images/admin_console.png)

![](images/api_key.png)

![](images/customer_id.png)

##### Load Environment Variables

In [1]:
import os
import json
import time
import requests
from dotenv import load_dotenv

load_dotenv()

api_url = "https://api.prod.alltrue-be.com"  # Replace with the actual base URL of your Guardium AI Security tenant
api_key = os.getenv("API_KEY")  # Set your API key as an environment variable
customer_id = os.getenv("CUSTOMER_ID")  # Replace with your actual customer ID

#### Get a JWT Token

In [2]:
def get_jwt_token(api_key):
    endpoint = f"{api_url}/v1/auth/issue-jwt-token"
    headers = {"X-API-Key": f"{api_key}"}
    response = requests.post(endpoint, headers=headers)
    response.raise_for_status()
    return response.json()["access_token"]


jwt_token = get_jwt_token(api_key)

#### Function to make API requests

In [3]:
def make_api_request(endpoint, token: str, method="GET",  data=None, params=None):
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    url = f"{api_url}{endpoint}"

    response = requests.request(method, url, headers=headers, params=params, json=data)
    response.raise_for_status()
    return response.json()

#### Make API Requests

##### Get Organization and Project Names

![](images/org_proj_name.png)

##### Get Organization and Project IDs from Names

In [4]:
organization = os.getenv("ORGANIZATION_NAME")
project = os.getenv("PROJECT_NAME")

# Input desired API here
api = f"/v1/admin/customers/{customer_id}/organizations/projects"

# Call API function
response = make_api_request(api, token=jwt_token, method="GET", data=None, params=None)
response_dict = json.loads(json.dumps(response))

# Extract Org ID
org_info = [org for org in response_dict["organizations"] if org.get('organization_name') == organization]
org_id = org_info[0]['organization_id']

# Extract Project ID
project_id = [proj.get('project_id') for proj in org_info[0].get('projects') if proj.get('project_name') == project][0]

# Print Output
print('org_id:\t\t', org_id)
print('project_id:\t', project_id)

org_id:		 74127eaf-b945-4055-a688-d8e552358e03
project_id:	 35ed0fe6-e8b4-4d82-834d-0cb239ef24e9


### Create LLM Endpoint

![](images/_configure.png)
![](images/_base_url.png)
![](images/_headers.png)

In [5]:
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_KEY")

api = f"/v1/inventory/customer/{customer_id}/resources"
method = "POST"

data = {
  "resources": [
    {
      "resource_type": "CustomLlmEndpoint",
      "resource_data": {
        "api_key": AZURE_OPENAI_API_KEY,
        "endpoint_identifier": "Airline Azure OpenAI 1 Proxy",
        "type": "custom",
        "pentest_connection_details": {
          "pentest_url": "https://api.8951b5a1.prod.alltrue-be.com/custom/openai/deployments/gpt-35-turbo/chat/completions?api-version=2024-10-21",
          "method": "POST",
          "body": {
            "model": "gpt-35-turbo",
            "messages": [
              {
                "role": "user",
                "content": "<<PROMPT>>"
              }
            ]
          },
          "headers": {
            "x-alltrue-llm-endpoint-identifier": "Airline Azure OpenAI 1",
            "x-alltrue-llm-base-url": "https://ai-hui6981ai199195655194.openai.azure.com",
            "x-alltrue-llm-proxy-type": "azure-openai",
            "x-alltrue-llm-path-matchers": "[\".*\"]",
            "x-alltrue-llm-domain-matchers": "[\".*\"]",
            "api-key": AZURE_OPENAI_API_KEY
          },
          "response_jsonpaths": [
            "$.choices[0].message.content"
          ],
          "response_type": "json",
          "response_error_values": [
            "Internal Error"
          ],
          "customer_scripts": []
        }
      },
      "project_ids": [
        project_id
      ],
      "technology_types": [
        "custom-llm-endpoint"
      ]
    }
  ]
}

# Call Create Resource Endpoint
response = make_api_request(api, token=jwt_token, method=method, data=data, params=None)
resource_instance_id = response["added_resources"][0]["resource_instance_id"]
print('resource_instance_id:\t', resource_instance_id)

resource_instance_id:	 ab80cb79-f27f-425e-95ff-f20710c260cf


### Retrieve Scan Template ID

##### Get Template Name

![](images/template_name.png)

In [6]:
template_name = 'Do Anything Now'

api = f"/v2/llm-pentest/customer/{customer_id}/templates"

method = "GET"
response = make_api_request(api, token=jwt_token, method=method, data=None, params=None)

template_id=[template["llm_pentest_scan_template_id"] for template in response["llm_pentest_scan_templates"] if template["name"] == template_name][0]
print(template_id)

4a550ae4-a7b2-42fe-9e3c-4b0f3be5c375


### Run Pentest

In [7]:
api = f"/v2/llm-pentest/customer/{customer_id}/start-pentest"
method = "POST"
data = {
    "resource_instance_id": resource_instance_id,
    "llm_pentest_scan_template_id": template_id,
    "description": "CI-triggered pentest",
    "pentest_connection_details": {}
}

response = make_api_request(api, token=jwt_token, method=method, data=data)
job_id = response["job_id"]
llm_pentest_scan_execution_id = response["llm_pentest_scan_execution_id"]

print(json.dumps(response, indent=4))

{
    "job_id": "64103a7f-1fd5-4b66-a14a-9cc927f90122",
    "report_job_id": "fbeefeb9-480d-43cb-bbad-bf565333716c",
    "status": "RUNNING",
    "llm_pentest_scan_execution_id": "46090eb6-a190-46ee-b5ff-a25b7839d103"
}


### Wait for Pentest to Complete

In [8]:
status = 'RUNNING'

while status == 'RUNNING':
    api = f"/v2/llm-pentest/job-status/{job_id}"
    method = "GET"
    response = make_api_request(api, token=jwt_token, method=method, data=None, params=None)
    status = response["status"]
    print(status)
    time.sleep(1)

RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
COMPLETED


### Download CSV Report

In [9]:
endpoint = f"/v2/llm-pentest/customer/{customer_id}/executions/{llm_pentest_scan_execution_id}/download-csv"
method = "POST"


def download_csv(endpoint, token: str, method="GET",  data=None, params=None):
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    url = f"{api_url}{endpoint}"

    response = requests.request(method, url, headers=headers, params=params, json=data)
    # Check if the request was successful
    if response.status_code == 200:

        with open(f'{time.time()}.csv', "wb") as f:
            f.write(response.content)
        print("CSV saved successfully!")
    else:
        print(f"Error: {response.status_code} - {response.text}")


# Call Download CSV Endpoint
download_csv(endpoint, token=jwt_token, method=method, data=None, params=None)

CSV saved successfully!
